In [ ]:
# 从shiningspectrum中导入pretreatment
from shiningspectrum import pretreatment
#从shiningspectrum中导入database
from shiningspectrum import database
#导入peak_processing
from shiningspectrum import peak_processing
import os
import matplotlib.pyplot as plt
import numpy as np

import math
import pickle
#import matplotlib.pyplot as plt
#import numpy as np
#from ramannoodles import spectrafit
from ramannoodles import shoyu
from ramannoodles import peakidentify

In [ ]:
data_path=os.getcwd()+"\\Prepare incoming data"

file_data1 = database.read_file(data_path, "氯仿.txt")#67-66-3
file_data2 = database.read_file(data_path, "甲苯.txt")#108-88-3

list_spectrum1 = database.data_extraction(file_data1)
list_spectrum2 = database.data_extraction(file_data2)

list_spectrum_compound = [list_spectrum1[0],list(np.array(list_spectrum1[1])+np.array(list_spectrum2[1]))]

plt.figure(figsize=(10,10))
plt.plot(list_spectrum_compound[0],list_spectrum_compound[1],"k-",label="list_spectrum_compound")
plt.plot(list_spectrum1[0],list_spectrum1[1],"r-",label="list_spectrum1")

plt.plot(list_spectrum2[0],list_spectrum2[1],"b-",label="list_spectrum2")
plt.legend()

In [ ]:
def shining2noodles(all_spectrum):
    list_of_compounds = []
    for key in list(all_spectrum.keys()):
        x = np.array(all_spectrum[key][0])
        y = np.array(all_spectrum[key][1])
        
        list_of_compounds.append(({"title":key,"x":x,"y":y}))
        
    return list_of_compounds

In [ ]:
all_spectrum = database.read_all("raman_database")

list_of_compounds = shining2noodles(all_spectrum)

#unknown_x, unknown_y = shoyu.combine_spectra(list_of_compounds[0], list_of_compounds[3], plot = True)#原版
unknown_x, unknown_y = shiningnoodles.combine_spectra(list_of_compounds[0], list_of_compounds[3])#shining重构

In [ ]:
#y_data = subtract_baseline(unknown_y)#原版
#x_data = unknown_x#原版
x_data, y_data, y_base = pretreatment.autbaseline(unknown_x, unknown_y, deg=4, max_it=200,tol=None)#shining重构

unknow_compound = {"title":"unkonw","x":x_data,"y":y_data}

peaks = peak_detect(x_data, y_data,prominence=y_data.mean())[0]
mod, pars = set_params(peaks)

In [ ]:
out = model_fit(x_data, y_data, mod, pars)#慢

In [ ]:
fit_peak_data = export_fit_data(out)#快
peak_centers = []
peak_sigma = []
peak_ampl = []
# peak_height = []
print("开始循环")
for i, _ in enumerate(fit_peak_data):
    # peak_fractions.append(fit_peak_data[i][0])
    # if we ever need lorentzian fraction we can add it
    # right now it may break other functions 
    peak_sigma.append(fit_peak_data[i][1])
    peak_centers.append(fit_peak_data[i][2])
    peak_ampl.append(fit_peak_data[i][3])
    # peak_height.append(fit_peak_data[i][5])
xmin = min(x_data)
xmax = max(x_data)

In [ ]:
unknown_peaks=peak_centers

known_compound_list = list_of_compounds
known_compound_peaks = []
assignment_matrix = []
precision = 0.03
for i, _ in enumerate(known_compound_list):
    known_compound_peaks.append(
        compound_report(known_compound_list[i])[0])
    print("The peaks that we found for "
          + str(known_compound_list[i]['title']) + " are: ")
    print(known_compound_peaks[i])
    
    assignment_matrix.append(compare_unknown_to_known(unknown_peaks,
                                                      known_compound_peaks[i],
                                                      precision))



In [ ]:
#Ok, so that generates a full association matrix that contains everything
#we need to assign peaks.
#Now, let's go through and actually assign text to peaks.
#print("spectrafit.data_report成功")

unknown_peak_assignments = peak_position_comparisons(unknown_peaks,
                                                     known_compound_peaks,
                                                     known_compound_list,
                                                     assignment_matrix)
print(unknown_peak_assignments)

percentages = percentage_of_peaks_found(known_compound_peaks,
                                        assignment_matrix,
                                        known_compound_list)



In [ ]:
plot=True
if plot:
    plotting_peak_assignments(unknown_x, unknown_y, unknown_peaks,
                              unknown_peak_assignments,percentages)

#percentages = percentage_of_peaks_found(known_compound_peaks,
#                                        assignment_matrix,
#                                        known_compound_list)
print(percentages)